## 5.3.- Servidor WEB

En esta seccion se describira como implemetar un servidor web en el SOC ESP8266 que permita, en este caso prender o apagar un led. Se utilizara una libreria especialmente desarrollada con este fin que se denomina ESP8266WebServer.

### 5.3.1 Librería ESP8266WebServer
 
Un servidor web es una computadora que almacena los archivos que componen un sitio web (ej.  documentos HTML , imágenes, hojas de estilos CSS y archivo JavaScript) y los entrega a otro dispositivo que le solicita un servicio. Se desarrollo para el ESP8266 una libreria que permite implementar un servidor web. Se denomina libreria esp8266WebServer. Los metodos que se utilizan se muestran a continuacion.   


| Metodo          | Sintaxis                   |  Funcion      |
|---              |---                         |---            |
|ESP8266WebServer |ESP8266WebServer server(80) | Crea el objeto server |
|begin            | server.begin()             | Inicializa el servidor|
|close            | server.close()             | Apaga el servidor     |
|stop             | server.stop()              | Apaga el servidor     |
| on              | server.on("/", FuncionM);  |  indicamos una función que emitira una respuesta del servidor cuando un navegador solicita la URL |
|addHandler       | server. addHandler();      |              |
| onNotFound      | server.onNotFound();       |              |
| onFileUpload    | server.onFileUpload();     |              |
| send            | server.send(code,tipo_contenido,contenido)| Envia las respuestas solicitadas|
|                 |                                           |code:200, 404 |
|                 |                                           |tipo_contenido:"text/plain","image/png"|
|                 |                                           |contenido: Debe ser una cadena de caracteres |
|streamFile       |server.streamFile(file, GetContentType(path));| Permite cargar una pagina completa |
|handleClient     |server.handleClient();      |  Maneja las conexiones entrantes|


### 5.3.2.- Programa Servidor WEB

El servidor web tiene que enviar una pagina Web a quien quiere comunicarse con el. Esta pagina Web debe almacenarse en el SOC Esp8266. En este caso se almacena en la pagina de la memoria flash del dispositivo. Se utilizara el metodo **PROGMEM**. Despues se puede pasar a la RAM cuando hace falta.

```
const char MAIN_page[] PROGMEM = R"=====( pagina web )======"
```
Este programa se denominara internet1. Se debe crear desde el IDE de Arduino. Se recomienda utilizar los archivos config.h y wifi.hpp de la seccion anterior. En este sentido se debe copiar los mismos al nuevo directorio creado con nombre internet1. El entorno de programacion quedara con los siguiente archivos:
- Internet1.ino
- config.h
- wifi.hpp

  Los cuales pasan a ser descriptos.




#### Pagina WEB
La pagina web que presentara el  servidor web se muestra a continuación.   

```
//---------------------------------------------------------------
// Texto en HTML

const char MAIN_page[] PROGMEM = R"=====(  
<!DOCTYPE html>  
<html>  
<body>  
<center>  
<h1>Programa para prender/apagar un LED </h1><br>
Apretar para <a href="/prender"><button>Prender LED</button></a><br>
Apretar para <a href="/apagar"><button>Apagar LED</button></a><br>
<hr>
</center>

</body>
</html>
)=====";
```
El aspecto de la misma se muestra abajo.

El tag **<a>** se utiliza para enviar indicaciones al servidor web en este caso prenderLed y apagarLed.


//---------------------------------------------------------------
// Texto en HTML
<!DOCTYPE html>  
<html>  
<body>  
<center>  
<h1>Programa para prender/apagar un LED </h1><br>
Apretar para <a href="/prender"><button>Prender LED</button></a><br>
Apretar para <a href="/apagar"><button>Apagar LED</button></a><br>
<hr>
</center>

</body>
</html>

### config.h
Se presenta el codigo completo del archivo config.h

In [ ]:
const char* red_WIFI = "red";  
const char* clave = "password";  


//---------------------------------------------------------------
// Texto en HTML

const char MAIN_page[] PROGMEM = R"=====(  
<!DOCTYPE html>  
<html>  
<body>  
<center>  
<h1>Programa para prender/apagar un LED </h1><br>
Apretar para <a href="/prender"><button>Prender LED</button></a><br>
Apretar para <a href="/apagar"><button>Apagar LED</button></a><br>
<hr>
</center>

</body>
</html>
)=====";


#### wifi.hpp
Se describe a continuación el archivo wifi.hpp donde se describe la coneccion a internet y el servidor web que controla el led integrado en la placa.

##### Funcion ConectarWIFI
La funcion conectarWifi conecta el SOC esp8266 con Internet a travez de una red WIFI. En ese sentido se utiliza: **WIFI.begin** para realizar la conexión, **WIFI.status()** para determinar en que momento se realizo la conexion y finalmente **WIFI.localIP()** donde se muestra el numero IP del SOC ESP8266 en la red.

```
void ConectarWifi()
{

  WiFi.begin(red_WIFI, clave);
  Serial.print("Conectando");
  while (WiFi.status() != WL_CONNECTED) {
    delay(500);
    Serial.print(".");
  }
   Serial.println();
  Serial.print("Conectado, dirección IP: ");
  Serial.println(WiFi.localIP());
  Serial.println("");
}
```
##### iniciarServer()
La funcion iniciar server regula el funcionamiento completo del servidor web. Cuando un usuario (cliente) quiere acceder a la pagina. **/**. Se ejecuta la funcion **controlRaiz**. Si se apreta el boton prender led. Usando el Tag <A> se envia al dispositivo la orden /Prender que ejecuta la función prender. Lo mismo para Apagar. Finalmente se inicia el servidor utilizando **server.begin**. Previamente se debe cargar el constructor de la clase que se muestra a continuacion.
```
void iniciar_server() {

  server.on("/", controlRaiz);
  server.on("/Prender", controlLedPrender);  //Rutina mueve el brazo
  server.on("/Apagar", controlLedApagar);    //Rutina mueve el brazo
  server.begin();
}
```
##### controlRaiz()
Es la funcion que carga la pagina WEB cuando es requerida por algun usuario (cliente).
```
void controlRaiz() {  
 Serial.println("Pagina original ");  
 String s = MAIN_page; //Read HTML contents  
 server.send(200, "text/html", s); //Send web page  
}  
```
##### ControlLedPrender()

Se utiliza para encender el LEd que se encuentra en la placa del SOC ESP8266y luego recargar la pagina WEB.
```
void controlLedPrender() {   
 Serial.println("LED prendido");  
 digitalWrite(BUILTIN_LED,LOW); //LED is connected in reverse  
 server.send(200, "text/html", "El LED esta prendido");  
}  
```
##### ControlLedApagar()

Se utiliza para apagar el LEd que se encuentra en la placa del SOC ESP8266 y luego recargar la pagina WEB.

```
void controlLedApagar() {   
 Serial.println("LED apagado");  
 digitalWrite(BUILTIN_LED,HIGH); //LED off  
 server.send(200, "text/html", "El LED esta apagado");   
}
```

Se presenta el codigo completo del archivo wifi.hpp.   

In [ ]:
void ConectarWifi()
{

  WiFi.begin(red_WIFI, clave);
  Serial.print("Conectando");
  while (WiFi.status() != WL_CONNECTED) {
    delay(500);
    Serial.print(".");
  }
   Serial.println();
  Serial.print("Conectado, dirección IP: ");
  Serial.println(WiFi.localIP());
  Serial.println("");
}
ESP8266WebServer server(80);
void controlRaiz() {  
 Serial.println("Pagina original ");  
 String s = MAIN_page; //Read HTML contents  
 server.send(200, "text/html", s); //Send web page  
}  
void controlLedPrender() {   
 Serial.println("LED prendido");  
 digitalWrite(BUILTIN_LED,LOW); //LED is connected in reverse  
 server.send(200, "text/html", "El LED esta prendido");  
}  
void controlLedApagar() {   
 Serial.println("LED apagado");  
 digitalWrite(BUILTIN_LED,HIGH); //LED off  
 server.send(200, "text/html", "El LED esta apagado");   
}
void iniciar_server() {

  server.on("/", controlRaiz);
  server.on("/Prender", controlLedPrender);  //Rutina mueve el brazo
  server.on("/Apagar", controlLedApagar);    //Rutina mueve el brazo
  server.begin();
}

#### internet1.ino

Este es el programa principal en donde se configura la red wifi utilizando **conectarWifi()** se activa el servidor WEB con **iniciar_server()** y se espera la conexion de un usuario en loop con **server.handleClient()**


In [ ]:
#include <ESP8266WiFi.h>
#include <ESP8266WebServer.h>
#include "config.h"
#include "wifi.hpp"
void setup() {  
  //  Configurar la consola  
  Serial.begin(115200);
pinMode(BUILTIN_LED,OUTPUT);
  Serial.println();  
  // Configura led de la placa  
  pinMode(LED_BUILTIN, OUTPUT);  
  // Configura el WIFI  
  conectarWifi();  
  iniciar_server();  
}  

void loop() {  
  server.handleClient();  
}  